## 0. 设置参数

In [1]:
import pandas as pd
from rdkit import Chem
from tqdm import tqdm
import numpy as np



sv_path = "/gaozhangyang/experiments/MotifRetro/data/uspto_50k/"
train = pd.read_csv("/gaozhangyang/experiments/MotifRetro/data/uspto_50k/raw_train.csv")
valid = pd.read_csv("/gaozhangyang/experiments/MotifRetro/data/uspto_50k/raw_val.csv")
test = pd.read_csv("/gaozhangyang/experiments/MotifRetro/data/uspto_50k/raw_test.csv")
data = pd.concat([train,valid, test], axis=0)
tasks = []
for idx in tqdm(range(len(data))):
    target, source = data.iloc[idx]["reactants>reagents>production"].split(">>")
    tasks.append([target, source])
    
    
    
# sv_path = "/gaozhangyang/experiments/MotifRetro/data/uspto_hard/"
# data = pd.read_csv("/gaozhangyang/experiments/MotifRetro/data/uspto_hard/USPTOHard.csv")
# tasks = []
# for idx in tqdm(range(len(data))):
#     target, source = data.loc[idx,"substrates"], data.loc[idx,"product"]
#     tasks.append([target, source])


# sv_path = "/gaozhangyang/experiments/MotifRetro/data/uspto_full/"
# data = pd.read_csv("/gaozhangyang/experiments/MotifRetro/data/uspto_full/USPTOFull.csv")
# tasks = []
# for idx in tqdm(range(len(data))):
#     target, source = data.loc[idx,"rxn_smiles"].split(">>")
#     tasks.append([target, source])
    

100%|██████████| 50016/50016 [00:03<00:00, 13221.98it/s]


## 1.提取fragment

In [4]:
import sys; sys.path.append("/gaozhangyang/experiments/MotifRetro")
import pandas as pd
from rdkit import Chem
import json
from copy import copy
from src.utils.feat_utils import atom_to_edit_tuple, get_bond_tuple, fix_incomplete_mappings, reac_to_canonical, renumber_atoms_for_mapping, mark_reactants
from rdkit.Chem import Draw
from src.utils.chem_utils import remove_am
from src.feat.reaction_actions import StopAction, AddMotifAction
from tqdm import tqdm
import numpy as np
from src.feat.featurize_gzy_psvae import ReactionSampleGenerator as ReactionSampleGenerator_gzy

from joblib import Parallel, delayed, cpu_count
from tqdm import tqdm

def pmap_multi(pickleable_fn, data, n_jobs=None, verbose=1, desc=None, **kwargs):
    """

    Parallel map using joblib.

    Parameters
    ----------
    pickleable_fn : callable
        Function to map over data.
    data : iterable
        Data over which we want to parallelize the function call.
    n_jobs : int, optional
        The maximum number of concurrently running jobs. By default, it is one less than
        the number of CPUs.
    verbose: int, optional
        The verbosity level. If nonzero, the function prints the progress messages.
        The frequency of the messages increases with the verbosity level. If above 10,
        it reports all iterations. If above 50, it sends the output to stdout.
    kwargs
        Additional arguments for :attr:`pickleable_fn`.

    Returns
    -------
    list
        The i-th element of the list corresponds to the output of applying
        :attr:`pickleable_fn` to :attr:`data[i]`.
    """
    if n_jobs is None:
        n_jobs = cpu_count() - 1

    results = Parallel(n_jobs=n_jobs, verbose=verbose, timeout=None)(
    delayed(pickleable_fn)(*d, **kwargs) for i, d in tqdm(enumerate(data),desc=desc)
    )

    return results

# utils
action_vocab = json.load(open("/gaozhangyang/experiments/MotifRetro/dataset_code/action_vocab.json", 'r'))

props = action_vocab['prop2oh']
prop2oh = {'atom': {}, 'bond': {}}

for type_key in prop2oh.keys():
    oh_dict = props[type_key]
    for key, values in oh_dict.items():
        converted_values = {}
        for prop_val, val_oh in values.items():
            try:
                prop_val = int(prop_val)
            except ValueError:
                pass
            converted_values[prop_val] = val_oh
        prop2oh[type_key][key] = converted_values

action_vocab['prop2oh'] = prop2oh



# visualization
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D, MolsToGridImage

drawOptions = rdMolDraw2D.MolDrawOptions()
drawOptions.prepareMolsBeforeDrawing = False
drawOptions.bondLineWidth = 4
drawOptions.minFontSize = 12


def prepare_mol(mol, new_am):
    highlight_idx = []
    for i, atom in enumerate(mol.GetAtoms()):
        am = atom.GetAtomMapNum()
        if am in new_am:
            highlight_idx.append(i)
            
    try:
        mol_draw = rdMolDraw2D.PrepareMolForDrawing(mol)
    except Chem.KekulizeException:
        mol_draw = rdMolDraw2D.PrepareMolForDrawing(mol, kekulize=False)
        Chem.SanitizeMol(mol_draw, Chem.SANITIZE_ALL ^ Chem.SANITIZE_KEKULIZE)
    
    
    return mol_draw, highlight_idx

def plot_states(states, target_mol, source_mol):
    target_am = set([atom.GetAtomMapNum() for atom in  target_mol.GetAtoms()])
    source_am = set([atom.GetAtomMapNum() for atom in  source_mol.GetAtoms()])
    new_am = list(target_am - source_am)

    mol_list = []
    highlightAtomLists = []
    for one in states:
        mol, highlight = prepare_mol(one, new_am)
        mol_list.append(mol)
        highlightAtomLists.append(highlight)

    return MolsToGridImage(mol_list, molsPerRow=5,  subImgSize=(500, 500), drawOptions=drawOptions, highlightBondLists = highlightAtomLists)


from src.utils.chem_utils import MultiElement
from src.utils.retro_utils import preprocess_mols, get_synthons, match_synthons_reactants, get_frag_mol




def handle_per_task(target, source):
    try:
        target_mol, source_mol = preprocess_mols(target, source)

        motifretro_sample_generator = ReactionSampleGenerator_gzy(Chem.rdchem.RWMol(source_mol), target_mol, keep_actions_list=None, action_vocab=action_vocab, use_motif_action=True, only_get_synthon=True)

        synthons = get_synthons(motifretro_sample_generator)

        synthons = MultiElement(synthons).mols
        reactants = MultiElement(target_mol).mols
        reactants = match_synthons_reactants(synthons, reactants)
        frag_mol_list = []
        for k in range(len(reactants)):
            frag_mol = get_frag_mol(synthons[k], reactants[k])
            frag_mol_list.append((frag_mol, source_mol, target_mol))
            # frag_smiles_list.append(Chem.MolToSmiles(remove_am(frag_mol)))
        return frag_mol_list
    except:
        print(source)
        print(target)
        print()
        return None
    


In [5]:
frag_mol_list_raw = pmap_multi(handle_per_task, [(target, source) for (target, source) in tasks])

[Parallel(n_jobs=126)]: Using backend LokyBackend with 126 concurrent workers.
315it [00:19, 55.86it/s][Parallel(n_jobs=126)]: Done 198 tasks      | elapsed:    6.8s
[Parallel(n_jobs=126)]: Done 548 tasks      | elapsed:    8.8s
[Parallel(n_jobs=126)]: Done 998 tasks      | elapsed:   10.7s
[Parallel(n_jobs=126)]: Done 1548 tasks      | elapsed:   12.6s
[Parallel(n_jobs=126)]: Done 2198 tasks      | elapsed:   15.0s


[C:1]([CH2:2][CH:3]=[CH2:4])([C:5](=[O:6])[O:7][CH2:8][CH3:9])([CH:10]([F:11])[F:12])[NH2:13]
O=C([C:1]([CH2:2][CH:3]=[CH2:4])([C:5](=[O:6])[O:7][CH2:8][CH3:9])[CH:10]([F:11])[F:12])[NH2:13]



[Parallel(n_jobs=126)]: Done 2948 tasks      | elapsed:   17.6s


[CH3:1][C:2](=[O:3])[c:8]1[cH:7][cH:6][c:5]([OH:4])[cH:10][cH:9]1
[CH3:1][C:2](=[O:3])[O:4][c:5]1[cH:6][cH:7][cH:8][cH:9][cH:10]1



[Parallel(n_jobs=126)]: Done 3798 tasks      | elapsed:   20.3s
[Parallel(n_jobs=126)]: Done 4748 tasks      | elapsed:   22.9s
[Parallel(n_jobs=126)]: Done 5798 tasks      | elapsed:   26.0s
[Parallel(n_jobs=126)]: Done 6948 tasks      | elapsed:   27.7s


...

[c:1]1([NH:33][C@H:18]2[CH2:31][CH2:30][N:21]([C@H:20]3[CH2:19][CH2:17][C@@H:28]([N:29]([CH3:22])[CH:32]([CH3:24])[CH3:25])[CH2:26][C@H:23]3[CH2:27][CH2:16][CH3:15])[C:34]2=[O:35])[cH:2][cH:3][n:4][c:5]2[cH:6][cH:7][c:8]([C:9]([F:10])([F:11])[F:12])[cH:13][c:14]12
Cl[c:1]1[cH:2][cH:3][n:4][c:5]2[cH:6][cH:7][c:8]([C:9]([F:10])([F:11])[F:12])[cH:13][c:14]12.[CH3:15][CH2:16][CH2:17][C@@H:18]1[CH2:19][C@H:20]([N:21]([CH3:22])[CH:23]([CH3:24])[CH3:25])[CH2:26][CH2:27][C@@H:28]1[N:29]1[CH2:30][CH2:31][C@H:32]([NH2:33])[C:34]1=[O:35]



[Parallel(n_jobs=126)]: Done 44748 tasks      | elapsed:  1.9min


[C:1]([NH:3][C@@H:2]([C:5]([CH3:6])([CH3:7])[CH3:35])[C:28]([N:26]([CH3:27])[C@H:30](/[CH:21]=[C:19](/[C:17]([O:16][CH2:15][CH3:14])=[O:18])[CH3:20])[CH:23]([CH3:24])[CH3:25])=[O:29])(=[O:13])[C@H:22]([NH:31][CH3:4])[C:32]([c:8]1[cH:9][cH:10][cH:11][s:12]1)([CH3:33])[CH3:34]
O[C:1]([C@@H:2]([NH:3][CH3:4])[C:5]([CH3:6])([CH3:7])[c:8]1[cH:9][cH:10][cH:11][s:12]1)=[O:13].[CH3:14][CH2:15][O:16][C:17](=[O:18])/[C:19]([CH3:20])=[CH:21]/[C@H:22]([CH:23]([CH3:24])[CH3:25])[N:26]([CH3:27])[C:28](=[O:29])[C@@H:30]([NH2:31])[C:32]([CH3:33])([CH3:34])[CH3:35]

[CH2:1]=[CH:2][C:3]([NH:4][C:6]([CH3:5])([CH3:7])[CH3:8])=[O:9]
[CH2:1]=[CH:2][C:3]#[N:4].[CH3:5][C:6]([CH3:7])([CH3:8])[OH:9]



[CH3:1][C:2]([CH3:3])([CH3:4])[NH:6][C:7](=[O:5])[CH:8]1[CH2:9][NH:10][CH2:11][CH2:12][NH:13]1
[CH3:1][C:2]([CH3:3])([CH3:4])[OH:5].[N:6]#[C:7][CH:8]1[CH2:9][NH:10][CH2:11][CH2:12][NH:13]1



[Parallel(n_jobs=126)]: Done 47798 tasks      | elapsed:  2.0min
50016it [02:02, 409.37it/s]
[Parallel(n_jobs=126)]: Done 50016 out of 50016 | elapsed:  2.0min finished


In [6]:
frag_mol_list_raw = sum([one for one in frag_mol_list_raw if one is not None],[])

frag_source_target_list = [(Chem.MolToSmiles(remove_am(one[0])), Chem.MolToSmiles(one[1]), Chem.MolToSmiles(one[2])) for one in frag_mol_list_raw]

print(f"valid frag mol:{len(frag_source_target_list)}")

valid frag mol:85544


In [7]:
frag_smiles_list = [frag_smi for (frag_smi,source_smi, target_smi) in frag_source_target_list]
frag_smiles_list_splitted = sum([one.split(".") for one in frag_smiles_list], [])

# 我们只考虑单点编辑的情况

def canonical_smiles_per_task(smi):
    # 设置sanitize=False防止MolFromSmiles时自动将C变成c
    return Chem.MolToSmiles(Chem.MolFromSmiles(smi, sanitize=False))

frag_smiles_list_splitted = [ canonical_smiles_per_task(one)  for one in frag_smiles_list_splitted]



from collections import Counter
frag_smiles_count = dict(Counter(frag_smiles_list_splitted))
frag_smiles_count = sorted(frag_smiles_count.items(),key = lambda x:x[1],reverse = True)
frag_smiles_count = {key:val for key, val in frag_smiles_count}
json.dump(frag_smiles_count, open(sv_path+"frag_smiles_count.json", "w"), indent=4)
 
with open(sv_path+"frag_smiles.txt","w") as f:
    f.writelines("\n".join(list(frag_smiles_count.keys())))

In [8]:
data_with_frag = pd.DataFrame(columns=["frag","source", "target"])
for i, (frag,source, target) in enumerate(frag_source_target_list):
    data_with_frag.loc[i,"frag"] = frag
    data_with_frag.loc[i,"source"] = source
    data_with_frag.loc[i,"target"] = target
data_with_frag.to_csv("/gaozhangyang/experiments/MotifRetro/data/uspto_50k/data_with_frag.csv")

In [28]:
for idx, one in enumerate(frag_smiles_list):
    for smi in one.split("."):
        if canonical_smiles_per_task(smi) == "O=c1*c(=O)c2ccccc12":
            print(idx)
            break

368
409
948
1296
1541
1637
2705
2722
3458
3956
4383
5127
6101
7453
7578
7878
8203
8308
8314
8945
9409
10583
10717
11437
11523
12776
13048
13207
13269
14570
15651
18407
18759
20203
20214
20754
20772
20866
21411
22325
23671
24376
24891
24958
25516
26657
26658
26702
26832
26965
28561
29157
29949
30928
31018
31514
31928
32191
32337
32393
32902
33254
35595
37185
37888
38493
41887
42164
42753
42853
43519
44912
45478
46905
47963
48499
49631
49632
50971
51249
51293
51665
51803
52357
52931
53683
53718
53780
53903
54248
54551
56198
57092
57235
57340
57729
59882
60643
61188
61217
61373
62284
63336
63382
63506
64281
64500
65007
65010
65933
66860
66863
68895
68912
69096
73058
73360
75295
75622
75834
76027
76348
76591
77480
77580
77735
78628
79306
79860
80192
80202
80461
80878
81114
81204
81896
82106
82121
82367
82605
82647
84780
